In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

testset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:05<00:00, 29.5MB/s]


In [3]:
class CNN(nn.Module):
    def __init__(self, num_classes, activation):
        super(CNN, self).__init__()

        self.act = activation

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            self.act,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            self.act,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            self.act,
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            self.act,
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [4]:
import torch.nn.init as init

def initialize_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if init_type == "xavier":
                init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                init.kaiming_uniform_(m.weight, nonlinearity="relu")
            elif init_type == "random":
                init.normal_(m.weight, mean=0, std=0.02)

            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


In [5]:
activations = {
    "relu": nn.ReLU(),
    "tanh": nn.Tanh(),
    "leaky_relu": nn.LeakyReLU(0.01)
}

optimizers = {
    "sgd": optim.SGD,
    "adam": optim.Adam,
    "rmsprop": optim.RMSprop
}


In [6]:
def evaluate(model, testloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total


In [ ]:
best_accuracy = 0
num_epochs = 40   # 🔒 FIXED: sab models ke liye 40 epochs

for act_name, act_fn in activations.items():
    for init_type in ["xavier", "kaiming", "random"]:
        for opt_name, opt_fn in optimizers.items():

            print("\n===================================")
            print(f"Activation = {act_name}, Init = {init_type}, Optimizer = {opt_name}")
            print("===================================")

            model = CNN(num_classes=10, activation=act_fn)
            initialize_weights(model, init_type)

            criterion = nn.CrossEntropyLoss()
            optimizer = opt_fn(model.parameters(), lr=0.001)

            # 🔹 Training (40 epochs for EVERY combination)
            for epoch in range(num_epochs):
                model.train()
                epoch_loss = 0.0

                for images, labels in trainloader:
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item()

                avg_loss = epoch_loss / len(trainloader)
                print(f"Epoch [{epoch+1}/{num_epochs}]  Loss: {avg_loss:.4f}")

            # 🔹 Evaluation after full 40 epochs
            accuracy = evaluate(model, testloader)
            print(f"Final Test Accuracy: {accuracy:.2f}%")

            # 🔹 Save best model overall
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                torch.save(model.state_dict(), "best_cnn_cifar10.pth")
                print("✅ Best model saved")



Activation = relu, Init = xavier, Optimizer = sgd
Epoch [1/40]  Loss: 2.0742
